<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import sqlite3
from tabulate import tabulate

db = 'site.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    with sqlite3.connect(db) as connection:
      cursor = connection.cursor()
      cursor.execute(f'SELECT * FROM {tablename}')
      results = cursor.fetchall()
      headers = []
      for i in range(len(cursor.description)):
        headers.append(cursor.description[i][0])
      print(tablename)
      print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
      print("\n")

  for table in tables:
    show(table)

show_tables()

users
+------+--------+---------+
|  id  |  name  |  email  |
+======+========+=========+
+------+--------+---------+


chats
+------+--------+-----------+
|  id  |  name  |  user_id  |
+======+========+===========+
+------+--------+-----------+


messages
+------+-----------+-----------+-----------+
|  id  |  user_id  |  chat_id  |  message  |
+======+===========+===========+===========+
+------+-----------+-----------+-----------+




In [96]:
def create_db():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("""CREATE TABLE users (
                      id INTEGER PRIMARY KEY,
                      name TEXT,
                      email TEXT
                    );""")

    cursor.execute("""CREATE TABLE chats (
                      id INTEGER PRIMARY KEY,
                      name TEXT,
                      user_id INTEGER,
                      FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
                    );""")

    cursor.execute("""CREATE TABLE messages (
                      id INTEGER PRIMARY KEY,
                      user_id INTEGER,
                      chat_id INTEGER,
                      message TEXT,
                      FOREIGN KEY (user_id) REFERENCES users (id),
                      FOREIGN KEY (chat_id) REFERENCES chats (id) ON DELETE CASCADE
                    );""")

    connection.commit()

# create_db()


def insert():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("""INSERT INTO users (name, email) VALUES ('John Doe', 'johndoe@example.com');""")
    cursor.execute("""INSERT INTO chats (name, user_id) VALUES ('General', 1);""")
    cursor.execute("""INSERT INTO messages (user_id, chat_id, message) VALUES (1, 1, 'Hello, world!');""")
    connection.commit()

# insert()



def delete():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("""DELETE FROM users WHERE name = 'John Doe';""")
    cursor.execute("""DELETE FROM chats WHERE name = 'General';""")
    cursor.execute("""DELETE FROM messages WHERE message = 'Hello, world!';""")
    connection.commit()

delete()